In [581]:
from requests import request
import requests

def volume(x, y, z): return x*y*z
def density(mass, volume): return mass/volume

compartments = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/compartment").json()
containers = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/container").json()
shipments = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/shipment").json()
luggage = requests.get(url="https://af-cargo-api-cargo.azuremicroservices.io/api/luggage").json()

for container in containers:
    container["volume"]  = volume(container["length"], container["width"], container["height"])

compartment_conf = [
    {"PMC":0,"PAG":0,"AKE":10},
    {"PMC":0,"PAG":0,"AKE":10},
    {"PMC":0,"PAG":0,"AKE":8},
    {"PMC":0,"PAG":0,"AKE":8},
    {"PMC":0,"PAG":0,"AKE":1},
]

no_of_first_class_luggage = luggage["nbFirstClassLuggage"]
no_of_second_class_luggage = + luggage["nbLuggage"]
avg_wt = luggage["avgWeight"] 

for shipment in shipments:
    shipment["volume"]  = volume(shipment["length"], shipment["width"], shipment["height"])
    shipment["density"]  = density(shipment["weight"], shipment["volume"])

shipments = sorted(shipments, key=lambda k: k['density'], reverse=True)

In [582]:
def knapsack(n, weight, count, values, weights):
    dp = [[[0] * (weight + 1) for _ in range(n + 1)] for _ in range(count + 1)]
    for z in range(1, count + 1):
        for y in range(1, n + 1):
            for x in range(weight + 1):
                if weights[y - 1] <= x:
                    dp[z][y][x] = max(dp[z][y - 1][x],
                                      dp[z - 1][y - 1][x - weights[y - 1]] + values[y - 1])
                else:
                    dp[z][y][x] = dp[z][y - 1][x]

    return dp[-1][-1][-1]

In [583]:
result = [
    {
        "compartmentId": 1,
        "containersWithShipments": [],
        "containersWithLuggage": []
    },
    {
        "compartmentId": 2,
        "containersWithShipments": [],
        "containersWithLuggage": []
    },
    {
        "compartmentId": 3,
        "containersWithShipments": []
    },
    {
        "compartmentId": 4,
        "containersWithShipments": []
    },
    {
        "compartmentId": 5,
        "containersWithLuggage": []
    }
]

In [584]:
def place_luggage(no_of_luggage):
    available_luggage = no_of_luggage

    if available_luggage >= 38:
        result[4]["containersWithLuggage"].append({
                    "containerType": "AKE",
                    "nbOfLuggage": 38
                    }
                )
        available_luggage -= 38
    else:
        result[4]["containersWithLuggage"].append({
                    "containerType": "AKE",
                    "nbOfLuggage": available_luggage
                    }
                )
        return
     
    print(available_luggage)
    for i in [1, 2, 3, 4]:
        available_ake = compartment_conf[i-1]["AKE"]
        available_pmc = compartment_conf[i-1]["PMC"]
        available_pag = compartment_conf[i-1]["PAG"]
        available_wt = compartments[i - 1]["maxWeight"]

        for j in range(available_ake):
            compartments[i - 1]["maxWeight"] = available_wt
            if (available_luggage <= 0 or available_wt <= 0):
                break
            luggage_to_add = 38 if available_luggage >= 38 else available_luggage
            # Wt constraint
            wt_to_add = luggage_to_add * 20
            if available_wt < wt_to_add:
                luggage_to_add = available_wt//20
                wt_to_add = luggage_to_add*20

            compartment_conf[i-1]["AKE"] -= 1

            available_luggage -= luggage_to_add
            available_wt -= wt_to_add
            result[i-1]["containersWithLuggage"].append({
                "containerType": "AKE",
                "nbOfLuggage": luggage_to_add
            }
            )


# firstclass
place_luggage(no_of_first_class_luggage+no_of_second_class_luggage)

# Secondclass
# place_luggage(no_of_second_class_luggage)


321


In [585]:
result

[{'compartmentId': 1,
  'containersWithShipments': [],
  'containersWithLuggage': [{'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 38},
   {'containerType': 'AKE', 'nbOfLuggage': 17}]},
 {'compartmentId': 2,
  'containersWithShipments': [],
  'containersWithLuggage': []},
 {'compartmentId': 3, 'containersWithShipments': []},
 {'compartmentId': 4, 'containersWithShipments': []},
 {'compartmentId': 5,
  'containersWithLuggage': [{'containerType': 'AKE', 'nbOfLuggage': 38}]}]

In [586]:
compartment_conf

[{'PMC': 0, 'PAG': 0, 'AKE': 1},
 {'PMC': 0, 'PAG': 0, 'AKE': 10},
 {'PMC': 0, 'PAG': 0, 'AKE': 8},
 {'PMC': 0, 'PAG': 0, 'AKE': 8},
 {'PMC': 0, 'PAG': 0, 'AKE': 1}]

In [587]:
def place_shipment():
    idx = 0

    for i in [4, 3, 2, 1]:
        available_ake = compartment_conf[i-1]["AKE"]
        available_pmc = compartment_conf[i-1]["PMC"]
        available_pag = compartment_conf[i-1]["PAG"]
        available_wt = compartments[i - 1]["maxWeight"]

        for j in range(available_ake):
            if (len(shipments) <= 0):
                print("debugger 1")
                break
            remaining_volume = containers[2]["volume"]*0.89
            if available_wt < shipments[idx]["weight"]:
                print("debugger 2")
                break

            shipments_to_add = []
            remaining_container_wt = 1588

            while (True):
                if (len(shipments) <= 0):
                    print("debugger 3")
                    break
                if available_wt < shipments[idx]["weight"]:
                    print("debugger 4")
                    break

                print(i, available_wt, shipments[idx]
                      ["weight"], remaining_container_wt)

                if remaining_volume >= shipments[idx]["volume"]:
                    if (len(shipments) <= 0 or idx >= len(shipments)-1):
                        print("debugger 5")
                        break
                    if available_wt < shipments[idx]["weight"] or remaining_container_wt < shipments[idx]["weight"]:
                        print("debugger 6")
                        idx += 1
                        continue

                    print("->", remaining_container_wt,
                          "->", shipments[idx]["weight"])
                    remaining_container_wt = remaining_container_wt - \
                        shipments[idx]["weight"]
                    available_wt = available_wt - shipments[idx]["weight"]
                    remaining_volume = remaining_volume - \
                        shipments[idx]["volume"]
                    shipment = shipments.pop(idx)
                    shipments_to_add.append(shipment["id"])

                else:
                    break
            idx = 0
            if (len(shipments_to_add) > 0):
                result[i-1]["containersWithShipments"].append({
                    "containerType": "AKE",
                    "shipments": shipments_to_add
                })


place_shipment()


4 10955 345.0 1588
-> 1588 -> 345.0
4 10610.0 345.0 1243.0
-> 1243.0 -> 345.0
4 10265.0 345.0 898.0
-> 898.0 -> 345.0
4 9920.0 345.0 553.0
-> 553.0 -> 345.0
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 345.0 208.0
debugger 6
4 9575.0 3480.0 208.0
debugger 6
4 9575.0 3480.0 208.0
debugger 6
4 9575.0 3480.0 208.0
debugger 6
4 9575.0 3480.0 208.0
debugger 6
4 9575.0 3480.0 208.0
debugger 6
4 9575.0 3480.0 208.0
debugger 6
4 9575.0 714.9 208.0
debugger 6
4 9575.0 714.9 208.0
debugger 6
4 9575.0 714.9 208.0
debugger 6
4 9575.0 30.0 208.0
-> 208.0 -> 30.0
4 9545.0 30.0 178.0
-> 178.0 -> 30.0
4 9515.0 30.0 148.0
-> 148.0 -> 30.0
4 9485.0 780.0 118.0
debugger 6
4 9485.0 780.0 118.0
debugger 6
4 9485.0 780.0 118.0
d

In [588]:
import json

json.dumps(result)

'[{"compartmentId": 1, "containersWithShipments": [{"containerType": "AKE", "shipments": [19]}], "containersWithLuggage": [{"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 38}, {"containerType": "AKE", "nbOfLuggage": 17}]}, {"compartmentId": 2, "containersWithShipments": [{"containerType": "AKE", "shipments": [71]}, {"containerType": "AKE", "shipments": [72]}, {"containerType": "AKE", "shipments": [73]}, {"containerType": "AKE", "shipments": [74]}, {"containerType": "AKE", "shipments": [22, 23, 31, 32]}, {"containerType": "AKE", "shipments": [33, 34, 35]}, {"containerType": "AKE", "shipments": [36, 37, 38]}, {"containerType": "AKE", "shipments": [24, 25, 17]}], "containersWithLuggage":

In [589]:
shipments

[{'id': 39,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 40,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 41,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 42,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 43,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 44,
  'awb': '057-08862022',
  'height': 80,
  'width': 120,
  'length': 103,
  'weight': 3480.0,
  'volume': 988800,
  'density': 0.003519417475728155},
 {'id': 20,
  'awb': '074-55